# Packages

In [1]:
from utils import *

# Construction of the Admittance Matrix

In [2]:
shunt_admittance = {
   (2, 4): 0 + 1.72j,
    (2, 5): 0 + 0.88j,
    (4, 5): 0 + 0.44j,
}

impedance = {
    (1, 5): 0.00150 + 0.02j,
    (3, 4): 0.00075 + 0.01j,
    (2, 4): 0.0090 + 0.1j,
    (2, 5): 0.0045 + 0.050j,
    (4, 5): 0.00225 + 0.025j,
}
nodes = sorted(set(i for par in impedance for i in par))
node_map = {no: idx for idx, no in enumerate(nodes)}
Ybus = compute_y_bus(impedance,shunt_admittance)
df_Ybus = pd.DataFrame(Ybus, index=[f"Bus {i}" for i in nodes], columns=[f"Bus {i}" for i in nodes])
df_Ybus = df_Ybus.round(4)
df_Ybus

,Bus 1,Bus 2,Bus 3,Bus 4,Bus 5
Bus 1,3.7290-49.7203j,0.0000+ 0.0000j,0.0000+ 0.0000j,0.0000+ 0.0000j,-3.7290+ 49.7203j
Bus 2,0.0000+ 0.0000j,2.6783-28.4590j,0.0000+ 0.0000j,-0.8928+ 9.9197j,-1.7855+ 19.8393j
Bus 3,0.0000+ 0.0000j,0.0000+ 0.0000j,7.4580-99.4406j,-7.4580+ 99.4406j,0.0000+ 0.0000j
Bus 4,0.0000+ 0.0000j,-0.8928+ 9.9197j,-7.4580+99.4406j,11.9219-147.9589j,-3.5711+ 39.6786j
Bus 5,-3.7290+49.7203j,-1.7855+19.8393j,0.0000+ 0.0000j,-3.5711+ 39.6786j,9.0856-108.5782j


# Power Flow (Newton-Raphson Method)

In [7]:
V = np.array([1.0, 1.0, 1.05,1.0,1.0])
theta = np.radians([0.0, 0.0, 0.0, 0.0, 0.0])
P_spec = np.array([0,-8,4.4,0,0])
Q_spec = np.array([0,-2.8,-0.4,0,0])

slack = [0]
pv = [2]
pq = [1,3,4]

tol = 1e-6
max_iter = 200

for it in range(max_iter):
    P_calc, Q_calc = compute_power(V, theta, Ybus)
    dpv = P_spec[pv] - P_calc[pv]
    dpq = P_spec[pq] - P_calc[pq]
    dP = np.concatenate([dpv, dpq])
    dQ = Q_spec[pq] - Q_calc[pq]
    mismatch = np.concatenate([dP, dQ])

    if np.max(np.abs(mismatch)) < tol:
        print(f'Converged in {it} iterations!')
        break

    J = compute_jacobian(V, theta, Ybus, pv, pq)
    dx = np.linalg.solve(J, mismatch)

    for i, idx in enumerate(pv + pq):
        theta[idx] += dx[i]

    for i, idx in enumerate(pq):
        V[idx] += dx[len(pv) + len(pq) + i]


voltages_df = pd.DataFrame({
    'Magnitude (pu)': [round(v, 4) for v in V],
    'Angle (deg)': [round(np.degrees(t), 4) for t in theta]
}, index=[f'Bus {i+1}' for i in range(len(V))])

print("Final voltages:")
display(voltages_df)

Converged in 107 iterations!
Final voltages:


,Magnitude (pu),Angle (deg)
Bus 1,1.0000,0.0000
Bus 2,0.8338,-22.4064
Bus 3,1.0500,-0.5973
Bus 4,1.0193,-2.8340
Bus 5,0.9743,-4.5479


In [9]:
power_df = pd.DataFrame({
    'P (pu)': [round(p, 4) for p in P_calc],
    'Q (pu)': [round(q, 4) for q in Q_calc]
}, index=[f'Bus {i+1}' for i in range(len(P_calc))])

print("Power injections at all buses:")
display(power_df)

Power injections at all buses:


,P (pu),Q (pu)
Bus 1,3.9484,1.1428
Bus 2,-8.0000,-2.8000
Bus 3,4.4000,2.9748
Bus 4,-0.0000,0.0000
Bus 5,-0.0000,0.0000


In [8]:
losses = compute_loss(V, theta, impedance, shunt_admittance, node_map)

losses_df = pd.DataFrame({
    'P_loss (pu)': [round(p, 4) for _, _, p, _ in losses],
    'Q_loss (pu)': [round(q, 4) for _, _, _, q in losses]
}, index=[f'Line {i}–{k}' for i, k, _, _ in losses])

print("Losses per line:")
display(losses_df)

Losses per line:


,P_loss (pu),Q_loss (pu)
Line 1–5,0.0253,0.3379
Line 3–4,0.0192,0.2559
Line 2–4,0.1184,-0.1757
Line 2–5,0.1750,1.2213
Line 4–5,0.0104,-0.3218
